In [3]:
import requests
import pandas as pd
from IPython.display import display
import websockets
import asyncio
import json
import talib as ta

symbol='BTCUSDT'
interval='1d'

#API_KEY="193515abb42304bea46f6ac1d6752c67aef0247c0405bbdbb65c6a0bd637e5aa"
#API_SECRET="90cf096f9945f62c7128f0653808c2640e8c78b839e9a0c06f3bef93286d513f"

url=f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit=1000"
data=requests.get(url=url)
data=data.json()

class PlaceOrder:
    def _init_(self):
        self.API_KEY="193515abb42304bea46f6ac1d6752c67aef0247c0405bbdbb65c6a0bd637e5aa"
        self.API_SECRET="90cf096f9945f62c7128f0653808c2640e8c78b839e9a0c06f3bef93286d513f"
        self.BASE_URL="http://testnet.binance.vision"
    def buy(self):
        response=requests.get(BASE_URL + "/api/v3/time")
        server_time=response.json()["serverTime"]
        params = {
        'symbol': 'BTCUSDT',
        'side': 'BUY',
        'type': 'MARKET',
        'quantity': 0.001,
        'recvWindow': 5000,  # Time for the request to be valid in milliseconds
        'timestamp': server_time, 
            }
        selfparams["signature"]=create_signature(params,API_SECRET)
        headers={
        "X-MBX-APIKEY":API_KEY
        }
        response=requests.post(BASE_URL + endpoint , headers=headers , params=params )
        print(response.json())

    def sell(self):
        response=requests.get(BASE_URL + "/api/v3/time")
        server_time=response.json()["serverTime"]
        params = {
        'symbol': 'BTCUSDT',
        'side': 'BUY',
        'type': 'MARKET',
        'quantity': 0.001,
        'recvWindow': 5000,  # Time for the request to be valid in milliseconds
        'timestamp': server_time, 
            }
        selfparams["signature"]=create_signature(params,API_SECRET)
        headers={
        "X-MBX-APIKEY":API_KEY
        }
        response=requests.post(BASE_URL + endpoint , headers=headers , params=params )
        print(response.json())
    def create_signature(params,secret):
        query_string="&".join([f"{key}={params[key]}" for key in sorted (params)])
        return hmac.new(secret.encode("utf-8"),query_string.encode("utf-8"),hashlib.sha256).hexdigest()
        

def sarCalculation(data):
    global pos
    pos=='buy'
    sar=ta.SAR(data['High'],data['Low'],acceleration=0.05,maximum=0.5)
    if pos=='buy' and data['Close']>sar:
        ob=PlaceOrder()
        ob.buy()
    elif pos=='sell' and data['Close']<sar:
        ob=PlaceOrder()
        ob.sell()
    
    
    
columns=['Open Time','Open','High','Low','Close','Volume','Close Time','Base asset volume','Number of trades','Taker buy volume',
         'Taker buy base asset volume','Ignore']

df=pd.DataFrame(data,columns=columns)
df['Open']=df['Open'].astype(float)
df['Close']=df['Close'].astype(float)
df['High']=df['High'].astype(float)
df['Low']=df['Low'].astype(float)
df['Volume']=df['Volume'].astype(float)
df=df.set_index(pd.to_datetime(df['Open Time'],unit='ms'))

df=df.drop(['Base asset volume','Number of trades','Taker buy volume','Taker buy base asset volume','Ignore'],axis=1)

async def binance_kline(symbol,interval):
    global df
    url=f"wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}"
    async with websockets.connect(url)as wb:
        while True:
            message=await wb.recv()
            message=json.loads(message)
            #print(type(message))
            #print('kline closing time -->',message.get('k').get('T'))
            #print('last candle-->',df['Close Time'].iloc[-1])
            if message.get('k').get('T')==df['Close Time'].iloc[-1]:
                #update last row
                df.iloc[-1]=[
                    float(message.get('k').get('t')),
                    float(message.get('k').get('o')),
                    float(message.get('k').get('h')),
                    float(message.get('k').get('l')),
                    float(message.get('k').get('c')),
                    float(message.get('k').get('v')),
                    float(message.get('k').get('T')),
                    
                ]
                print("updating current row")
            else:
                #insert a new row
                d=pd.DataFrame({
                    "Open Time":float(message.get('k').get('t')),
                    "Open":float(message.get('k').get('o')),
                    "High":float(message.get('k').get('h')),
                    "Low":float(message.get('k').get('l')),
                    "Close":float(message.get('k').get('c')),
                    "Volume":float(message.get('k').get('v')),
                    "Close Time":float(message.get('k').get('T')),
                },index=[pd.Timestamp.now()])
                print("creating new row")
                df=pd.concat([df,d])
        sarCalculation(df)


               

            
display(df)
await binance_kline(symbol.lower(),interval)

,Open Time,Open,High,Low,Close,Volume,Close Time
Open Time,,,,,,,
2022-01-20,1642636800000,41660.00,43505.00,40553.31,40680.91,42330.339530,1642723199999
2022-01-21,1642723200000,40680.92,41100.00,35440.45,36445.31,88860.891999,1642809599999
2022-01-22,1642809600000,36445.31,36835.22,34008.00,35071.42,90471.338961,1642895999999
2022-01-23,1642896000000,35071.42,36499.00,34601.01,36244.55,44279.523540,1642982399999
2022-01-24,1642982400000,36244.55,37550.00,32917.17,36660.35,91904.753211,1643068799999
...,...,...,...,...,...,...,...
2024-10-11,1728604800000,60326.40,63417.56,60087.64,62540.00,23641.352090,1728691199999
2024-10-12,1728691200000,62539.99,63480.00,62487.23,63206.22,10911.301160,1728777599999
2024-10-13,1728777600000,63206.23,63285.72,62050.00,62870.02,11909.219950,1728863999999


updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row
updating current row


CancelledError: 

In [5]:

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from IPython.display import display

from backtesting.test import SMA, GOOG
import talib as ta

testing_df=df.drop(['Close Time','Open Time'],axis=1)
# display(testing_df)
#print(testing_df.columns)
#print(type(testing_df))

class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 7)
        self.ma2 = self.I(SMA, price, 15)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()

class RsiCross(Strategy):
    def init(self):
        price=self.data.Close
        self.rsi=self.I(ta.RSI,price)
        self.overbought=70
        self.oversell=30
    def next(self):
        if self.rsi<=self.oversell:
            self.buy()
        elif self.rsi>=self.overbought:
            self.sell()
        
class MARSI(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)
        self.overbought=70
        self.oversell=30
    def next(self):
        if crossover(self.ma1, self.ma2) or self.rsi<=self.oversell:
            self.buy()
        elif crossover(self.ma2, self.ma1) or self.rsi>=self.overbought:
            self.sell()

#SAR
class SARCross(Strategy):
    def init(self):
        self.high=self.data.High
        self.low=self.data.Low
        self.acceleration=0.04
        self.maximum=0.4
        self.sar=self.I(ta.SAR,self.high,self.low,acceleration=self.acceleration,maximum=self.maximum)
    def next(self):
        if crossover(self.data.Close,self.sar):
            self.buy()
        if crossover(self.sar,self.data.Close):
            self.sell()
        



bt = Backtest(testing_df, SARCross, commission=.002,
              exclusive_orders=True,cash=1000000)

#print(type(GOOG))
#print(GOOG.columns)
display(testing_df)
stats = bt.run()
print(stats)
bt.plot() 

,Open,High,Low,Close,Volume
2022-01-20,41660.00,43505.00,40553.31,40680.91,42330.339530
2022-01-21,40680.92,41100.00,35440.45,36445.31,88860.891999
2022-01-22,36445.31,36835.22,34008.00,35071.42,90471.338961
2022-01-23,35071.42,36499.00,34601.01,36244.55,44279.523540
2022-01-24,36244.55,37550.00,32917.17,36660.35,91904.753211
...,...,...,...,...,...
2024-10-11,60326.40,63417.56,60087.64,62540.00,23641.352090
2024-10-12,62539.99,63480.00,62487.23,63206.22,10911.301160
2024-10-13,63206.23,63285.72,62050.00,62870.02,11909.219950
2024-10-14,62870.02,66500.00,62457.81,66083.99,37669.952220


,Open,High,Low,Close,Volume
2022-01-20,41660.00,43505.00,40553.31,40680.91,42330.339530
2022-01-21,40680.92,41100.00,35440.45,36445.31,88860.891999
2022-01-22,36445.31,36835.22,34008.00,35071.42,90471.338961
2022-01-23,35071.42,36499.00,34601.01,36244.55,44279.523540
2022-01-24,36244.55,37550.00,32917.17,36660.35,91904.753211
...,...,...,...,...,...
2024-10-11,60326.40,63417.56,60087.64,62540.00,23641.352090
2024-10-12,62539.99,63480.00,62487.23,63206.22,10911.301160
2024-10-13,63206.23,63285.72,62050.00,62870.02,11909.219950
2024-10-14,62870.02,66500.00,62457.81,66083.99,37669.952220


Start                     2022-01-20 00:00:00
End                       2024-10-15 00:00:00
Duration                    999 days 00:00:00
Exposure Time [%]                        99.0
Equity Final [$]                1003090.58546
Equity Peak [$]                 1260741.85184
Return [%]                           0.309059
Buy & Hold Return [%]               61.511185
Return (Ann.) [%]                    0.112696
Volatility (Ann.) [%]               54.256438
Sharpe Ratio                         0.002077
Sortino Ratio                        0.003301
Calmar Ratio                         0.002728
Max. Drawdown [%]                  -41.308901
Avg. Drawdown [%]                   -9.036271
Max. Drawdown Duration      769 days 00:00:00
Avg. Drawdown Duration       90 days 00:00:00
# Trades                                  119
Win Rate [%]                        36.134454
Best Trade [%]                      33.767331
Worst Trade [%]                    -12.483884
Avg. Trade [%]                    

Row(id='2140', ...)

In [ ]:
#import time
import hashlib
import hmac
import requests

API_KEY="193515abb42304bea46f6ac1d6752c67aef0247c0405bbdbb65c6a0bd637e5aa"
API_SECRET="90cf096f9945f62c7128f0653808c2640e8c78b839e9a0c06f3bef93286d513f"
BASE_URL="http://testnet.binance.vision"


response=requests.get(BASE_URL + "/api/v3/time")
server_time=response.json()["serverTime"]
print(server_time)


# endpoint for placing orders 
endpoint="/api/v3/order"

params = {
    'symbol': 'BTCUSDT',
    'side': 'BUY',
    'type': 'MARKET',
    'quantity': 0.001,
    'recvWindow': 5000,  # Time for the request to be valid in milliseconds
    'timestamp': server_time, 
}

def create_signature(params,secret):
    query_string="&".join([f"{key}={params[key]}" for key in sorted (params)])
    return hmac.new(secret.encode("utf-8"),query_string.encode("utf-8"),hashlib.sha256).hexdigest()
params["signature"]=create_signature(params,API_SECRET)



headers={
    "X-MBX-APIKEY":API_KEY
}

response=requests.post(BASE_URL + endpoint , headers=headers , params=params )
print(response.json())